In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_all = sps.hstack([ICM1, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1234)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13643 ( 0.1%) Users that have less than 1 test interactions


In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {"num_factors": 29, "epochs": 50, "confidence_scaling": "log", "alpha": 0.001, "epsilon": 0.001, "reg": 0.01},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True}
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender

In [10]:
recommender_3 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender_4 = SLIM_BPR_Cython(URM_train)
recommender_1 = RP3betaRecommender(URM_train)
recommender_2 = ItemKNNCBFRecommender(URM_train, ICM_weighted)
recommender3 = IALSRecommender_Hybrid(URM_train, ICM_all)

model_init(recommender_3, 'SLIM_Pure', models_to_combine_best['SLIMER'])
model_init(recommender_4, 'SLIM_BPR', models_to_combine_best['SLIMBPR'])
model_init(recommender_1, 'RP3beta', models_to_combine_best['RP3beta'])
model_init(recommender_2, 'KNNweigh', models_to_combine_best['icm_weighted'])
model_init(recommender3, 'IALS_Hyb', models_to_combine_best['IALS_Hyb'])

recommender1 = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_3.W_sparse, recommender_4.W_sparse)
recommender1.fit(0.9876793526315789)

recommender2 = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_1.W_sparse, recommender_2.W_sparse)
recommender2.fit(0.9546136842105264)

ItemKNNCBFRecommender: ICM Detected 22 ( 0.1%) items with no features.
../models_temp/Similarity_Hybrid/SLIM_Pure.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIM_Pure'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/SLIM_BPR.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIM_BPR'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/RP3beta.zip
Model found!
RP3betaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/RP3beta'
RP3betaRecommender: Loading complete
../models_temp/Similarity_Hybrid/KNNweigh.zip
Model found!
ItemKNNCBFRecommender: Loading model from file '../models_temp/Similarity_Hybrid/KNNweigh'
ItemKNNCBFRecommender: Loading complete
../models_temp/Similarity_Hybrid/IALS_Hyb.zip
Model found!
IALSRecommender_Hybrid: Loading model from file '../models_temp/Similarity_Hybrid/IALS_Hyb'
IALSRecommender_Hy

In [11]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])

In [12]:
rank_models(evaluator_validation, [recommender1, recommender2, recommender3])

EvaluatorHoldout: Processed 13643 (100.0%) in 1.23 min. Users per second: 184
ItemKNNSimilarityHybridRecommender 0.25079121335553256
EvaluatorHoldout: Processed 13643 (100.0%) in 54.70 sec. Users per second: 249
ItemKNNSimilarityHybridRecommender 0.22369590661645253
EvaluatorHoldout: Processed 13643 (100.0%) in 9.26 sec. Users per second: 1474
IALSRecommender_Hybrid 0.2323499114029067


In [13]:
import matplotlib.pyplot as plt
import numpy as np

In [16]:
import numpy as np

def test_percentage(recommender_a, recommender_b, recommender_c, evaluator, high1, low1, high2, low2, high3, low3, step):
    recommender = ItemKNNScoresHybridMultipleRecommender(URM_train, recommender_a, recommender_b, recommender_c)
    results = []
    alp_space = np.linspace(high1, low1, step, True)
    beta_space = np.linspace(high2, low2, step, True)
    gamma_space = np.linspace(high3, low3, step, True)
    for alp in alp_space:
        for bet in beta_space:
            for gamma in gamma_space:
                recommender.fit(alp, bet, gamma)
                r_d, _ = evaluator.evaluateRecommender(recommender)
                print(alp, bet, gamma, ":", r_d.loc[10]['MAP'])
                results.append(r_d.loc[10]['MAP'])
    
    return alp_space, beta_space, gamma_space, results

In [18]:
alp_space, results = test_percentage(recommender1, recommender2, recommender3, evaluator_validation, 0.5, 0.8, 0.16, 0.25, 0.01, 0.1, 6)

_ = plt.figure(figsize=(16, 9))
plt.plot(results, alp_space, label='alpha')
plt.plot(results, beta_space, label='beta')
plt.plot(results, gamma_space, label='gamma')
plt.ylabel('MAP')
plt.xlabel('hyper')
plt.legend()
plt.show()

CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7462686567164178
ItemKNNSimilarityHybridRecommender with weight beta: 0.23880597014925373
IALSRecommender_Hybrid with weight gamma: 0.014925373134328358
EvaluatorHoldout: Processed 13643 (100.0%) in 2.09 min. Users per second: 109
0.5 0.16 0.01 : 0.2505787461213333
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7267441860465116
ItemKNNSimilarityHybridRecommender with weight beta: 0.23255813953488372
IALSRecommender_Hybrid with weight gamma: 0.040697674418604654
EvaluatorHoldout: Processed 13643 (100.0%) in 2.23 min. Users per second: 102
0.5 0.16 0.028000000000000004 : 0.2508880157159508
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7082152974504249
ItemKNNSimilarityHybridRecommender with weight beta: 0.22662889518413595
IALSRecommender_Hybrid with weight gamma: 0.06515580736543909
EvaluatorHoldout: Processed 13643 (100.0%) in 2.46 min. Users p

EvaluatorHoldout: Processed 13643 (100.0%) in 2.20 min. Users per second: 103
0.5 0.214 0.1 : 0.25070455923091
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6738544474393531
ItemKNNSimilarityHybridRecommender with weight beta: 0.3126684636118598
IALSRecommender_Hybrid with weight gamma: 0.013477088948787063
EvaluatorHoldout: Processed 13643 (100.0%) in 2.19 min. Users per second: 104
0.5 0.23199999999999998 0.01 : 0.25044406457640417
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6578947368421053
ItemKNNSimilarityHybridRecommender with weight beta: 0.3052631578947368
IALSRecommender_Hybrid with weight gamma: 0.0368421052631579
EvaluatorHoldout: Processed 13643 (100.0%) in 2.27 min. Users per second: 100
0.5 0.23199999999999998 0.028000000000000004 : 0.25083990394516026
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6426735218508998
ItemKNNSimilarityHybridRecommender with weight beta: 0.2982

0.56 0.178 0.082 : 0.25084971477902046
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6682577565632459
ItemKNNSimilarityHybridRecommender with weight beta: 0.21241050119331742
IALSRecommender_Hybrid with weight gamma: 0.11933174224343676
EvaluatorHoldout: Processed 13643 (100.0%) in 2.27 min. Users per second: 100
0.56 0.178 0.1 : 0.25084646001379945
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.731070496083551
ItemKNNSimilarityHybridRecommender with weight beta: 0.2558746736292428
IALSRecommender_Hybrid with weight gamma: 0.013054830287206266
EvaluatorHoldout: Processed 13643 (100.0%) in 2.12 min. Users per second: 107
0.56 0.196 0.01 : 0.25049587904257065
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7142857142857143
ItemKNNSimilarityHybridRecommender with weight beta: 0.25
IALSRecommender_Hybrid with weight gamma: 0.03571428571428572
EvaluatorHoldout: Processed 13643 (100.0%) in 2.26 mi

EvaluatorHoldout: Processed 13643 (100.0%) in 2.25 min. Users per second: 101
0.56 0.25 0.082 : 0.250892873140363
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6153846153846154
ItemKNNSimilarityHybridRecommender with weight beta: 0.2747252747252747
IALSRecommender_Hybrid with weight gamma: 0.10989010989010989
EvaluatorHoldout: Processed 13643 (100.0%) in 2.27 min. Users per second: 100
0.56 0.25 0.1 : 0.25074006205868776
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7848101265822784
ItemKNNSimilarityHybridRecommender with weight beta: 0.20253164556962025
IALSRecommender_Hybrid with weight gamma: 0.012658227848101266
EvaluatorHoldout: Processed 13643 (100.0%) in 2.13 min. Users per second: 107
0.62 0.16 0.01 : 0.2504949482786123
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7673267326732672
ItemKNNSimilarityHybridRecommender with weight beta: 0.198019801980198
IALSRecommender_Hybrid with w

EvaluatorHoldout: Processed 13643 (100.0%) in 2.21 min. Users per second: 103
0.62 0.214 0.082 : 0.2509094029265551
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6638115631691649
ItemKNNSimilarityHybridRecommender with weight beta: 0.2291220556745182
IALSRecommender_Hybrid with weight gamma: 0.10706638115631693
EvaluatorHoldout: Processed 13643 (100.0%) in 2.29 min. Users per second: 99
0.62 0.214 0.1 : 0.25095053396765016
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7192575406032483
ItemKNNSimilarityHybridRecommender with weight beta: 0.2691415313225058
IALSRecommender_Hybrid with weight gamma: 0.011600928074245941
EvaluatorHoldout: Processed 13643 (100.0%) in 2.19 min. Users per second: 104
0.62 0.23199999999999998 0.01 : 0.2504663243782216
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7045454545454546
ItemKNNSimilarityHybridRecommender with weight beta: 0.2636363636363636
IALSRecommen

...

EvaluatorHoldout: Processed 13643 (100.0%) in 3.28 min. Users per second: 69
0.68 0.25 0.064 : 0.2508326090826282
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6719367588932806
ItemKNNSimilarityHybridRecommender with weight beta: 0.24703557312252963
IALSRecommender_Hybrid with weight gamma: 0.08102766798418973
EvaluatorHoldout: Processed 13643 (100.0%) in 3.19 min. Users per second: 71
0.68 0.25 0.082 : 0.25091116265216595
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6601941747572816
ItemKNNSimilarityHybridRecommender with weight beta: 0.24271844660194175
IALSRecommender_Hybrid with weight gamma: 0.0970873786407767
EvaluatorHoldout: Processed 13643 (100.0%) in 3.15 min. Users per second: 72
0.68 0.25 0.1 : 0.25084124773562666
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.8131868131868132
ItemKNNSimilarityHybridRecommender with weight beta: 0.1758241758241758
IALSRecommender_Hybrid with w

EvaluatorHoldout: Processed 13643 (100.0%) in 1.19 min. Users per second: 191
0.74 0.214 0.064 : 0.25082452597936833
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7142857142857143
ItemKNNSimilarityHybridRecommender with weight beta: 0.20656370656370654
IALSRecommender_Hybrid with weight gamma: 0.07915057915057915
EvaluatorHoldout: Processed 13643 (100.0%) in 1.25 min. Users per second: 181
0.74 0.214 0.082 : 0.25087718976764756
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7020872865275142
ItemKNNSimilarityHybridRecommender with weight beta: 0.2030360531309298
IALSRecommender_Hybrid with weight gamma: 0.09487666034155598
EvaluatorHoldout: Processed 13643 (100.0%) in 1.26 min. Users per second: 181
0.74 0.214 0.1 : 0.2507962772931994
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7535641547861507
ItemKNNSimilarityHybridRecommender with weight beta: 0.23625254582484723
IALSRecommender_Hybrid

EvaluatorHoldout: Processed 13643 (100.0%) in 1.37 min. Users per second: 166
0.8 0.178 0.064 : 0.25097702292820806
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7547169811320755
ItemKNNSimilarityHybridRecommender with weight beta: 0.16792452830188678
IALSRecommender_Hybrid with weight gamma: 0.07735849056603773
EvaluatorHoldout: Processed 13643 (100.0%) in 1.38 min. Users per second: 165
0.8 0.178 0.082 : 0.2508920296355254
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7421150278293135
ItemKNNSimilarityHybridRecommender with weight beta: 0.16512059369202226
IALSRecommender_Hybrid with weight gamma: 0.09276437847866419
EvaluatorHoldout: Processed 13643 (100.0%) in 1.49 min. Users per second: 153
0.8 0.178 0.1 : 0.25088107861581505
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7952286282306164
ItemKNNSimilarityHybridRecommender with weight beta: 0.194831013916501
IALSRecommender_Hybrid wit

EvaluatorHoldout: Processed 13643 (100.0%) in 1.04 min. Users per second: 218
0.8 0.25 0.064 : 0.25085762045540055
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.7067137809187278
ItemKNNSimilarityHybridRecommender with weight beta: 0.22084805653710246
IALSRecommender_Hybrid with weight gamma: 0.07243816254416961
EvaluatorHoldout: Processed 13643 (100.0%) in 1.08 min. Users per second: 210
0.8 0.25 0.082 : 0.250905313382409
CURRENT CONFIGURATION:
ItemKNNSimilarityHybridRecommender with weight alpha: 0.6956521739130435
ItemKNNSimilarityHybridRecommender with weight beta: 0.21739130434782605
IALSRecommender_Hybrid with weight gamma: 0.08695652173913043
EvaluatorHoldout: Processed 13643 (100.0%) in 1.03 min. Users per second: 221
0.8 0.25 0.1 : 0.2508817766887847


ValueError: too many values to unpack (expected 2)